# COS-Halos Database (v1.0)

This Notebook is intended to record updates to the COS-Halos database.

Version History:

 * v1.0 -- 09 Feb 2016 by JXP

In [2]:
# imports
import pdb
from pyigm.cgm.cos_halos import COSHalos
from pyigm.cgm import cos_halos as pch

## Update the Megastructure

J0910+1014_34_46

has coverage of 5 SiII transitions yet the 
megastructure implies only 1:

IDL> help, megastruct[ii].ion[14,2], /str

    LOGNION         DOUBLE           12.957801
    SIGLOGNION      DOUBLE         0.081717335
    NFLG            LONG                 1
    NTRANS          LONG                 1
    BVAL            DOUBLE           0.0000000
    SIGBVAL         DOUBLE           0.0000000
    TRANS           STRUCT    -> TMPTRANS Array[16]


## Load FITS files from Megastructure

Generated on Dec 7, 2014 by JXP

This step will update the f-values (several lines were
using the old Morton 1991 reference) and associated column densities.

The synthesize_colm() method of AbsComponent is used for final columns.
These match the original to within 10% in all cases.

In [7]:
reload(pch)
cos_halos = pch.COSHalos()
cos_halos.load_mega()#test=True)

cos_halos: Reading /Users/xavier/Dropbox/COS-Halos-Data//Summary/J0226+0015.268_22.fits.gz
Skipping 1025.72 as NG for a line; flg=2
Skipping 972.537 as NG for a line; flg=2
cos_halos: Reading /Users/xavier/Dropbox/COS-Halos-Data//Summary/J0401-0540.67_24.fits.gz
Skipping 1025.72 as NG for a line; flg=2
cos_halos: Reading /Users/xavier/Dropbox/COS-Halos-Data//Summary/J0803+4332.306_20.fits.gz
cos_halos: Reading /Users/xavier/Dropbox/COS-Halos-Data//Summary/J0910+1014.242_34.fits.gz
Skipping 949.743 as NG for a line; flg=2
Skipping 937.803 as NG for a line; flg=13
Skipping 930.748 as NG for a line; flg=13
Skipping 926.226 as NG for a line; flg=15
Skipping 1304.37 as NG for a line; flg=10
cos_halos: Reading /Users/xavier/Dropbox/COS-Halos-Data//Summary/J0910+1014.34_46.fits.gz
Skipping 1025.72 as NG for a line; flg=13
cos_halos: Reading /Users/xavier/Dropbox/COS-Halos-Data//Summary/J0914+2823.41_27.fits.gz
Skipping 1190.42 as NG for a line; flg=2
cos_halos: Reading /Users/xavier/Dropbox/C

/Users/xavier/local/Python/linetools/linetools/isgm/abssystem.py:218: UserWarning: Input AbsComponent with does not match AbsSystem rules. Not appending
  warnings.warn('Input AbsComponent with does not match AbsSystem rules. Not appending')



> /Users/xavier/local/Python/pyigm/pyigm/cgm/cos_halos.py(203)load_single_fits()
-> if comp.flag_N != iont['FLG_CLM'][0]:
(Pdb) c
cos_halos: Reading /Users/xavier/Dropbox/COS-Halos-Data//Summary/J1342-0053.77_10.fits.gz
cos_halos: Reading /Users/xavier/Dropbox/COS-Halos-Data//Summary/J1419+4207.132_30.fits.gz
Skipping 1025.72 as NG for a line; flg=2
cos_halos: Reading /Users/xavier/Dropbox/COS-Halos-Data//Summary/J1435+3604.126_21.fits.gz
Skipping 949.743 as NG for a line; flg=2
Skipping 930.748 as NG for a line; flg=2
Skipping 926.226 as NG for a line; flg=2
cos_halos: Reading /Users/xavier/Dropbox/COS-Halos-Data//Summary/J1435+3604.68_12.fits.gz
Skipping 949.743 as NG for a line; flg=13
New colm for (8,1) and sys J1435+3604_68_12 is 0.854291 different from old
cos_halos: Reading /Users/xavier/Dropbox/COS-Halos-Data//Summary/J1437+5045.317_38.fits.gz
Skipping 949.743 as NG for a line; flg=4
Skipping 1302.17 as NG for a line; flg=13
Skipping 933.378 as NG for a line; flg=13
New colm f

/Users/xavier/local/Python/linetools/linetools/analysis/absline.py:125: RuntimeWarning: invalid value encountered in double_scalars
  lgvar = ((1.0 / (np.log(10.0)*N))**2)*sig_N**2
/Users/xavier/local/Python/pyigm/pyigm/cgm/cos_halos.py:198: RuntimeWarning: invalid value encountered in double_scalars
  if np.abs(comp.logN - float(iont['CLM'][0])) > 0.15:


## Write to JSON tarball

# FIX sig_logN for limits?

In [8]:
tarfil = cos_halos.cdir+'/cos-halos_systems.v1.0.tar.gz'

In [9]:
cos_halos.write_survey(tarfil)

Wrote: /Users/xavier/Dropbox/COS-Halos-Data//cos-halos_systems.v1.0.tar.gz


/Users/xavier/local/Python/pyigm/pyigm/cgm/cgmsurvey.py:70: UserWarning: Modify to write directly to tar file
  warnings.warn("Modify to write directly to tar file")


----

## Update $N_{\rm HI}$ measurements (Prochaska+16)

### Load JSON files

In [11]:
reload(pch)
cos_halos = pch.COSHalos()
cos_halos.load_sys(tfile=cos_halos.cdir+'/cos-halos_systems.v1.0.tar.gz')

Be patient, using /Users/xavier/Dropbox/COS-Halos-Data//cos-halos_systems.v1.0.tar.gz to load COS-Halos
Skipping a likely folder: CGM_JSON


### Load NHI measurements

In [12]:
import json
LLS_file = cos_halos.cdir + '/Updates/COS_Halos_LLS.json'
with open(LLS_file) as json_file:
    fdict = json.load(json_file)
#fdict

### Loop on systems

In [13]:
names = cos_halos.name

In [14]:
for key in fdict.keys():
    # Match
    mt = np.where(names == key)[0]
    if len(mt) != 1:
        raise ValueError("No match?!")
    # Fill in 
    if fdict[key]['flag_NHI'] == 1:
        cos_halos.cgm_abs[mt[0]].igm_sys.NHI = fdict[key]['fit_NHI'][0]
        cos_halos.cgm_abs[mt[0]].igm_sys.sig_NHI = (fdict[key]['fit_NHI'][2]-fdict[key]['fit_NHI'][1])/2.
        cos_halos.cgm_abs[mt[0]].igm_sys.flag_NHI = 1
    elif fdict[key]['flag_NHI'] == 2:
        cos_halos.cgm_abs[mt[0]].igm_sys.NHI = fdict[key]['fit_NHI'][1]
        cos_halos.cgm_abs[mt[0]].igm_sys.flag_NHI = 2
    elif fdict[key]['flag_NHI'] == 3:
        cos_halos.cgm_abs[mt[0]].igm_sys.NHI = fdict[key]['fit_NHI'][2]
        cos_halos.cgm_abs[mt[0]].igm_sys.flag_NHI = 3

### Write

In [15]:
tarfil = os.getenv('DROPBOX_DIR')+'COS-Halos-Data/cos-halos_systems.v1.1.tar.gz'

In [16]:
cos_halos.write_survey(tarfil)

Wrote: /u/xavier/Dropbox/COS-Halos-Data/cos-halos_systems.v1.1.tar.gz


----

## Modify a few column density measurements (Prochaska+16)

### Load JSON files

In [ ]:
cos_halos = COSHalos()
cos_halos.load_sys()

### Load bad ions

In [ ]:
import yaml
filename = cos_halos.cdir + '/Updates/COS_Halos_ions_updates.yaml'
with open(filename, 'r') as infile:
        up_ion_data = yaml.load(infile)

In [ ]:
up_ion_data

In [ ]:
names = cos_halos.name

In [ ]:
for key in up_ion_data.keys():
    # Match
    mt = np.where(names == key)[0]
    if len(mt) != 1:
        raise ValueError("No match?!")
    # Fill in 
    for ionkey in up_ion_data[key].keys():
        Zion = tuple([int(ii) for ii in ionkey.split(',')])
        #
        Zicomp = [comp.Zion for comp in cos_halos.cgm_abs[mt[0]].igm_sys._components]
        mtZi = Zicomp.index(Zion)
        # Set
        for att_key in up_ion_data[key][ionkey].keys():
            if att_key == 'flag_N':
                cos_halos.cgm_abs[mt[0]].igm_sys._components[mtZi].flag_N = up_ion_data[key][ionkey][att_key]
        print(cos_halos.cgm_abs[mt[0]].igm_sys._components[mtZi])

### Write

In [ ]:
tarfil = cos_halos.cdir+'/cos-halos_systems.v1.2.tar.gz'

In [ ]:
cos_halos.write_survey(tarfil)